In [1]:
import torch.nn as nn
import torch
import torchvision
from torchvision import transforms
from torchvision.transforms import ToTensor
from torch.utils import data
import matplotlib.pyplot as plt

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [3]:
class Block(nn.Module):
  def __init__(self, in_channel, out_channel, stride=1, identity_downsample=None):
    super().__init__()
    self.block = nn.Sequential(
        self._block(in_channel, out_channel, kernel_size=1, stride=1, padding=0),
        self._block(out_channel, out_channel, kernel_size=3, stride=stride, padding=1),
        self._block(out_channel, out_channel*4, kernel_size=1, stride=1, padding=0)
    )

    self.down_sample = identity_downsample
    self.relu = nn.ReLU()

  def forward(self, x: torch.tensor):
    identity = x
    x = self.block(x)

    if self.down_sample is not None:
      identity = self.down_sample(identity)
    
    x += identity
    return self.relu(x)

  def _block(self, in_channel, out_channel, kernel_size, stride, padding):
    block = nn.Sequential(
        nn.Conv2d(in_channel, out_channel, kernel_size, stride=stride, padding=padding),
        nn.BatchNorm2d(out_channel),
        nn.ReLU()
    )
    return block



class ResNet(nn.Module): #[3, 4, 6, 3]
  def __init__(self, block, img_channel, num_class, each_layer):
    super().__init__()
    self.in_channel = 64
    self.initial = nn.Sequential(
        nn.Conv2d(img_channel, self.in_channel, kernel_size=7, stride=2, padding=3),
        nn.BatchNorm2d(self.in_channel),
        nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
    )

    #ResNet Layer
    self.layer1 = self._make_layer(Block, each_layer[0], 64, 1)
    self.layer2 = self._make_layer(Block, each_layer[1], 128, 2)
    self.layer3 = self._make_layer(Block, each_layer[2], 256, 2)
    self.layer4 = self._make_layer(Block, each_layer[3], 512, 2)

    self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
    self.fc = nn.Linear(512*4, num_class)


  def forward(self, x: torch.tensor):
    x = self.initial(x)
    x = self.layer1(x)
    x = self.layer2(x)
    x = self.layer3(x)
    x = self.layer4(x)
    x = self.avgpool(x)
    x = x.reshape(x.shape[0],-1)
    x = self.fc(x)
    return x

  def _make_layer(self, Block, layers_num, out_channel, stride):
    identity_downsample = None
    layers = []

    if stride != 1 or self.in_channel != out_channel*4:
      identity_downsample = nn.Sequential(
          nn.Conv2d(self.in_channel, out_channel*4, kernel_size=1, stride=stride),
          nn.BatchNorm2d(out_channel*4)
      )
    
    layers.append(Block(self.in_channel, out_channel, stride=stride, identity_downsample=identity_downsample))
    self.in_channel = out_channel*4
    
    for i in range(layers_num - 1):
      layers.append(Block(self.in_channel, out_channel))
    
    return nn.Sequential(*layers)

In [4]:
x = torch.randn(1, 3, 224, 224)

In [5]:
def Res50(img_channel=3, num_class=10):
  return ResNet(Block, img_channel, num_class, each_layer=[3, 4, 6, 3])

def Res101(img_channel=3, num_class=10):
  return ResNet(Block, img_channel, num_class, each_layer=[3, 4, 23, 3])

def Res152(img_channel=3, num_class=10):
  return ResNet(Block, img_channel, num_class, each_layer=[3, 8, 23, 3])

In [6]:
model = Res50()
model(x)

tensor([[ 0.2380,  0.0645,  0.2857,  0.4857, -0.4785, -0.1943,  0.1695,  0.3103,
          1.5537,  0.8912]], grad_fn=<AddmmBackward0>)

In [11]:
class Block(nn.Module):
  def __init__(self, in_channel, out_channel, stride=1, identity_downsample=None):
    super().__init__()
    self.block = nn.Sequential(
        self._block(in_channel, out_channel, kernel_size=3, stride=1, padding=1),
        self._block(out_channel, out_channel, kernel_size=3, stride=stride, padding=1)
    )

    self.down_sample = identity_downsample
    self.relu = nn.ReLU()

  def forward(self, x: torch.tensor):
    identity = x
    x = self.block(x)

    if self.down_sample is not None:
      identity = self.down_sample(identity)
    
    x += identity
    return self.relu(x)

  def _block(self, in_channel, out_channel, kernel_size, stride, padding):
    block = nn.Sequential(
        nn.Conv2d(in_channel, out_channel, kernel_size, stride=stride, padding=padding),
        nn.BatchNorm2d(out_channel),
        nn.ReLU()
    )
    return block



class ResNet_kid(nn.Module): #[3, 4, 6, 3]
  def __init__(self, block, img_channel, num_class, each_layer):
    super().__init__()
    self.in_channel = 64
    self.initial = nn.Sequential(
        nn.Conv2d(img_channel, self.in_channel, kernel_size=7, stride=2, padding=3),
        nn.BatchNorm2d(self.in_channel),
        nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
    )

    #ResNet Layer
    self.layer1 = self._make_layer(Block, each_layer[0], 64, 1)
    self.layer2 = self._make_layer(Block, each_layer[1], 128, 2)
    self.layer3 = self._make_layer(Block, each_layer[2], 256, 2)
    self.layer4 = self._make_layer(Block, each_layer[3], 512, 2)

    self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
    self.fc = nn.Linear(512, num_class)


  def forward(self, x: torch.tensor):
    x = self.initial(x)
    x = self.layer1(x)
    x = self.layer2(x)
    x = self.layer3(x)
    x = self.layer4(x)
    x = self.avgpool(x)
    x = x.reshape(x.shape[0],-1)
    x = self.fc(x)
    return x

  def _make_layer(self, Block, layers_num, out_channel, stride):
    identity_downsample = None
    layers = []

    if stride != 1 or self.in_channel != out_channel:
      identity_downsample = nn.Sequential(
          nn.Conv2d(self.in_channel, out_channel, kernel_size=1, stride=stride),
          nn.BatchNorm2d(out_channel)
      )
    
    layers.append(Block(self.in_channel, out_channel, stride=stride, identity_downsample=identity_downsample))
    self.in_channel = out_channel
    
    for i in range(layers_num - 1):
      layers.append(Block(self.in_channel, out_channel))
    
    return nn.Sequential(*layers)

In [12]:
def Res18(img_channel=3, num_class=10):
  return ResNet_kid(Block, img_channel, num_class, each_layer=[2, 2, 2, 2])

def Res34(img_channel=3, num_class=10):
  return ResNet_kid(Block, img_channel, num_class, each_layer=[3, 4, 6, 3])

In [13]:
x = torch.randn(50, 3, 32, 32)
model = Res18()
y = model(x)

In [8]:
train_data = torchvision.datasets.CIFAR10(root='data',
                                               train=True,
                                               download=True,
                                               transform=ToTensor(),
                                               target_transform=None)

test_data = torchvision.datasets.CIFAR10(root='data',
                                               train=False,
                                               download=True,
                                               transform=ToTensor(),
                                               target_transform=None)

# train_data = torchvision.datasets.FashionMNIST(root='data',
#                                                train=True,
#                                                download=True,
#                                                transform=ToTensor(),
#                                                target_transform=None)

# test_data = torchvision.datasets.FashionMNIST(root='data',
#                                                train=False,
#                                                download=True,
#                                                transform=ToTensor(),
#                                                target_transform=None)

data_classes = train_data.classes
train_data.class_to_idx

train_batch = data.DataLoader(train_data, 50, True)
test_batch = data.DataLoader(test_data, 50, True)

loss_fn = torch.nn.CrossEntropyLoss()
opt = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)

def accuracy_fn(true, pred):
  correct = torch.eq(true, pred)
  return (torch.sum(correct)/len(true))*100

len(train_batch)


Files already downloaded and verified
Files already downloaded and verified


1000

In [18]:
from time import time
model2 = Res18().to(device)
torch.manual_seed(42)
start = time()

for epoch in range(1, 4):
  print(f'Epoch: {epoch}\n-------')
  train_loss = 0
  train_acc = 0
  for batch, (X, y) in enumerate(train_batch):
    X, y = X.to(device), y.to(device)
    
    model2.train()
    y_logit = model2(X)

    loss = loss_fn(y_logit, y)
    train_loss += loss

    accuracy = accuracy_fn(y, y_logit.argmax(dim=1))
    train_acc += accuracy

    opt.zero_grad()
    loss.backward()
    opt.step()
    if batch % 200 == 0 and batch != 0:
      print(f' batch: {batch} -- {batch*50} of {len(train_batch.dataset)} sample')
      
  train_loss /= len(train_batch)
  train_acc /= len(train_batch)
  print(f'\nTrain loss: {train_loss} -- Train accuracy: {train_acc}\n')

  model2.eval()
  with torch.inference_mode():
    test_loss = 0
    test_acc = 0
    for X_test, y_test in test_batch:
      X_test, y_test = X_test.to(device), y_test.to(device)
      y_test_logit = model2(X_test)

      test_loss += loss_fn(y_test_logit, y_test)
      test_acc += accuracy_fn(y_test, y_test_logit.argmax(dim=1))
    
    test_loss /= len(test_batch)
    test_acc /= len(test_batch)
    print(f'Test loss: {test_loss} -- Test accuracy: {test_acc}\n')
    print(f'Time: {time() - start:.3f} seconds\n')

Epoch: 1
-------


RuntimeError: ignored

In [17]:
torch.cuda.memory_allocated()

301656064